##**Import all neceassary packages**

In [ ]:
import numpy as np
import time
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import MultiStepLR, StepLR

from torchvision import datasets, transforms

from tqdm.notebook import tqdm as tqdm

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Model - Define GoogLeNet Model**

### conv_block, gridReduction, Inception x3, x5, x2, auxiliary classifier

In [ ]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.conv_layer(x)

class GridReduction(nn.Module):
    def __init__(self, in_fts, out_fts):
        super(GridReduction, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts, kernel_size=(3, 3), stride=(2, 2))
        )

        self.branch2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        )

    def forward(self, input_img):
        o1 = self.branch1(input_img)
        o2 = self.branch2(input_img)
        x = torch.cat([o1, o2], dim=1)
        return x

class Inceptionx3(nn.Module):
    def __init__(self, in_fts, out_fts):
        super(Inceptionx3, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[0], kernel_size=(1, 1), stride=(1, 1)),
            nn.Conv2d(in_channels=out_fts[0], out_channels=out_fts[0], kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.Conv2d(in_channels=out_fts[0], out_channels=out_fts[0], kernel_size=(3, 3), stride=(1, 1), padding=1)
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[1], kernel_size=(1, 1), stride=(1, 1)),
            nn.Conv2d(in_channels=out_fts[1], out_channels=out_fts[1], kernel_size=(3, 3), stride=(1, 1), padding=1),
        )
        self.branch3 = nn.Sequential(
            nn.AvgPool2d(kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[2], kernel_size=(1, 1), stride=(1, 1))
        )
        self.branch4 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[3], kernel_size=(1, 1), stride=(1, 1))
        )

    def forward(self, input_img):
        o1 = self.branch1(input_img)
        o2 = self.branch2(input_img)
        o3 = self.branch3(input_img)
        o4 = self.branch4(input_img)
        x = torch.cat([o1, o2, o3, o4], dim=1)
        return x

class Inceptionx5(nn.Module):
    def __init__(self, in_fts, out_fts, n=7):
        super(Inceptionx5, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[0], kernel_size=(1, 1), stride=(1, 1)),
            nn.Conv2d(in_channels=out_fts[0], out_channels=out_fts[0], kernel_size=(1, n), stride=(1, 1),
                      padding=(0, n // 2)),
            nn.Conv2d(in_channels=out_fts[0], out_channels=out_fts[0], kernel_size=(n, 1), stride=(1, 1),
                      padding=(n // 2, 0)),
            nn.Conv2d(in_channels=out_fts[0], out_channels=out_fts[0], kernel_size=(1, n), stride=(1, 1),
                      padding=(0, n // 2)),
            nn.Conv2d(in_channels=out_fts[0], out_channels=out_fts[0], kernel_size=(n, 1), stride=(1, 1),
                      padding=(n // 2, 0)),
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[1], kernel_size=(1, 1), stride=(1, 1)),
            nn.Conv2d(in_channels=out_fts[1], out_channels=out_fts[1], kernel_size=(1, n), stride=(1, 1),
                      padding=(0, n // 2)),
            nn.Conv2d(in_channels=out_fts[1], out_channels=out_fts[1], kernel_size=(n, 1), stride=(1, 1),
                      padding=(n // 2, 0)),
        )
        self.branch3 = nn.Sequential(
            nn.AvgPool2d(kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[2], kernel_size=(1, 1), stride=(1, 1))
        )
        self.branch4 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[3], kernel_size=(1, 1), stride=(1, 1))
        )

    def forward(self, input_img):
        o1 = self.branch1(input_img)
        o2 = self.branch2(input_img)
        o3 = self.branch3(input_img)
        o4 = self.branch4(input_img)
        x = torch.cat([o1, o2, o3, o4], dim=1)
        return x

class Inceptionx2(nn.Module):
    def __init__(self, in_fts, out_fts):
        super(Inceptionx2, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[0] // 4, kernel_size=(1, 1)),
            nn.Conv2d(in_channels=out_fts[0] // 4, out_channels=out_fts[0] // 4, kernel_size=(3, 3), stride=(1, 1),
                      padding=1)
        )
        self.subbranch1_1 = nn.Sequential(
            nn.Conv2d(in_channels=out_fts[0] // 4, out_channels=out_fts[0], kernel_size=(1, 3), stride=(1, 1),
                      padding=(0, 3 // 2))
        )
        self.subbranch1_2 = nn.Sequential(
            nn.Conv2d(in_channels=out_fts[0] // 4, out_channels=out_fts[1], kernel_size=(3, 1), stride=(1, 1),
                      padding=(3 // 2, 0))
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[2] // 4, kernel_size=(1, 1))
        )
        self.subbranch2_1 = nn.Sequential(
            nn.Conv2d(in_channels=out_fts[2] // 4, out_channels=out_fts[2], kernel_size=(1, 3), stride=(1, 1),
                      padding=(0, 3 // 2))
        )
        self.subbranch2_2 = nn.Sequential(
            nn.Conv2d(in_channels=out_fts[2] // 4, out_channels=out_fts[3], kernel_size=(3, 1), stride=(1, 1),
                      padding=(3 // 2, 0))
        )
        self.branch3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[4], kernel_size=(1, 1), stride=(1, 1))
        )
        self.branch4 = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts[5], kernel_size=(1, 1), stride=(1, 1))
        )

    def forward(self, input_img):
        o1 = self.branch1(input_img)
        o11 = self.subbranch1_1(o1)
        o12 = self.subbranch1_2(o1)
        o2 = self.branch2(input_img)
        o21 = self.subbranch2_1(o2)
        o22 = self.subbranch2_2(o2)
        o3 = self.branch3(input_img)
        o4 = self.branch4(input_img)
        x = torch.cat([o11, o12, o21, o22, o3, o4], dim=1)
        return x

# auxiliary classifier의 loss는 0.3이 곱해지고, 최종 loss에 추가합니다. 정규화 효과가 있습니다. 
class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()

        self.conv = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=5),
            conv_block(in_channels, 128, kernel_size=1),
        )

        self.fc = nn.Sequential(
            nn.Linear(5*5*128, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.7),
            nn.BatchNorm1d(num_features=1024),
            nn.Linear(1024, num_classes),
        )

    def forward(self,x):
        x = self.conv(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

### Declare Inception V3 model

In [ ]:
class GoogLeNet_v3(nn.Module):
    def __init__(self, aux_logits=True, num_classes=10, init_weights=True):
        super(GoogLeNet_v2, self).__init__()
        assert aux_logits == True or aux_logits == False
        self.aux_logits = aux_logits

        # conv_block takes in_channels, out_channels, kernel_size, stride, padding
        # Inception block takes out1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool

        # 3 X 299 X 299
        self.conv1 = conv_block(3, 32, kernel_size=3, stride=2)
        self.conv2 = conv_block(32, 32, kernel_size=3, stride=1)
        self.conv3 = conv_block(32, 64, kernel_size=3, stride=1, padding=1)
        # 64 X 147 X 147
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        # 64 X 73 X 73
        self.conv4 = conv_block(64, 80, kernel_size=3, stride=1)
        self.conv5 = conv_block(80, 192, kernel_size=3, stride=2)
        self.conv6 = conv_block(192, 288, kernel_size=3, stride=1, padding=1)
        # 288 X 35 X 35
        list_incept = [Inceptionx3(in_fts=288, out_fts=[96, 96, 96, 96]),
                       Inceptionx3(in_fts=4 * 96, out_fts=[96, 96, 96, 96]),
                       Inceptionx3(in_fts=4 * 96, out_fts=[96, 96, 96, 96])]

        self.inceptx3 = nn.Sequential(*list_incept)
        # 384 X 17 X 17
        self.grid_redn_1 = GridReduction(in_fts=4 * 96, out_fts=384)
        # 768 X 17 X 17
        if self.aux_logits:
            self.aux_classifier = InceptionAux(768, num_classes)
        else:
            self.aux_classifier = None
        # 768 X 17 X 17
        list_incept = [Inceptionx5(in_fts=768, out_fts=[160, 160, 160, 160]),
                       Inceptionx5(in_fts=4 * 160, out_fts=[160, 160, 160, 160]),
                       Inceptionx5(in_fts=4 * 160, out_fts=[160, 160, 160, 160]),
                       Inceptionx5(in_fts=4 * 160, out_fts=[160, 160, 160, 160]),
                       Inceptionx5(in_fts=4 * 160, out_fts=[160, 160, 160, 160])]

        self.inceptx5 = nn.Sequential(*list_incept)
        # 640 X 8 X 8
        self.grid_redn_2 = GridReduction(in_fts=4 * 160, out_fts=640)
        # 1280 X 8 X 8
        list_incept = [Inceptionx2(in_fts=1280, out_fts=[256, 256, 192, 192, 64, 64]),
                       Inceptionx2(in_fts=1024, out_fts=[384, 384, 384, 384, 256, 256])]

        self.inceptx2 = nn.Sequential(*list_incept)
        # 2048 X 8 X 8
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=1)
        # 2048 X 1 X 1
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, input_img):
        N = input_img.shape[0]
        x = self.conv1(input_img)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.inceptx3(x)
        x = self.grid_redn_1(x)
        aux_out = self.aux_classifier(x)
        x = self.inceptx5(x)
        x = self.grid_redn_2(x)
        x = self.inceptx2(x)
        x = self.avgpool(x)
        x = x.reshape(N, -1)
        x = self.fc(x)
        if self.aux_logits and self.training:
            return x, aux_out
        else:
            return x 

##**Utils**

In [ ]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt # 출력하는 소수의 자릿수
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

##**Cutout: Main Code for Applying Cutout data augmentation**

In [ ]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

##**Parameter Settings**

In [ ]:
dataset = 'cifar100' # cifar10 or cifar100
model = 'GoogLeNetV3' # resnet18, resnet50, resnet101, GoogLeNetV1
batch_size = 64  # Input batch size for training (default: 128)
epochs = 100 # Number of epochs to train (default: 200)
learning_rate = 1e-3 # Learning rate
data_augmentation = True # Traditional data augmentation such as augmantation by flipping and cropping?
sanity_check = False
path2weights = './drive/MyDrive/DeepLearning_competition/Internship/Week_2/GoogLeNet_V3.pth'    # route for model saving

cutout = True # Apply Cutout?
n_holes = 1 # Number of holes to cut out from image
length = 16 # Length of the holes

seed = 0 # Random seed (default: 0)
print_freq = 100
cuda = torch.cuda.is_available()
cudnn.benchmark = True  # Should make training should go faster for large models

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

test_id = dataset + '_' + model

##**Load and preprocess data**

In [ ]:
# Image Preprocessing
normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

# train
train_transform = transforms.Compose([])

train_transform.transforms.append(transforms.Resize((299, 299)))
if data_augmentation:
    train_transform.transforms.append(transforms.RandomCrop(299, 299))
    train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)

if cutout:
    train_transform.transforms.append(Cutout(n_holes=n_holes, length=length))

# test
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    normalize])

if dataset == 'cifar10':
    num_classes = 10
    train_dataset = datasets.CIFAR10(root='data/',
                                     train=True,
                                     transform=train_transform,
                                     download=True)

    test_dataset = datasets.CIFAR10(root='data/',
                                    train=False,
                                    transform=test_transform,
                                    download=True)
elif dataset == 'cifar100':
    num_classes = 100
    train_dataset = datasets.CIFAR100(root='data/',
                                      train=True,
                                      transform=train_transform,
                                      download=True)

    test_dataset = datasets.CIFAR100(root='data/',
                                     train=False,
                                     transform=test_transform,
                                     download=True)


# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          pin_memory=True,
                                          num_workers=2)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/
Files already downloaded and verified


##**Main Training**

In [ ]:
def train(train_loader, epoch, model, optimizer, criterion):
    batch_time = AverageMeter('Time', ':6.3f') # 소수 출력 형식지정
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, losses,
                             top1, top5, prefix="Epoch: [{}]".format(epoch))
    # switch to train mode
    model.train()
    best_model_wts = copy.deepcopy(model.state_dict())

    running_loss = 0.0
    len_data = len(train_loader.dataset)
    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        input = input.cuda()
        target = target.cuda()

        # compute output
        output = model(input) # output = (x, aux_out)

        loss_b = loss_batch(criterion, output, target, optimizer)
        running_loss += loss_b
        
        if sanity_check is True:
            break

        # measure accuracy and record loss, accuracy 
        acc1, acc5 = accuracy(output[0], target, topk=(1, 5))
        losses.update(loss_b, input.size(0))
        top1.update(acc1[0].item(), input.size(0))
        top5.update(acc5[0].item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            progress.print(i)
    
    loss = running_loss / len_data

    print('==> Train Accuracy: Acc@1 {top1.avg:.3f} || Acc@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))
    return top1.avg

def test(test_loader,epoch, model):
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')

    # switch to test mode
    model.eval()
    for i,(input,target) in enumerate(test_loader):
        input = input.cuda()
        target = target.cuda()

        output = model(input)
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        top1.update(acc1[0].item(), input.size(0))
        top5.update(acc5[0].item(), input.size(0))
    print('==> Test Accuracy:  Acc@1 {top1.avg:.3f} || Acc@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))
    return top1.avg

### Label smoothing

In [ ]:
class LabelSmoothing(nn.Module):
    def __init__(self, num_classes=1000, smoothing=0.1):
        super(LabelSmoothing, self).__init__()
        self.smoothing = smoothing
        self.k = num_classes

    def forward(self, target, pred):
        """
        pred (FloatTensor): [batch_size,n_classes]
        target (LongTensor): [batch_size]
        Ex- for batch_size=2
        target = tensor([[1],
                         [2]])
        pred = tensor([[0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
                      [0.0200, 0.0200, 0.0200, 0.0200, 0.0200]])
        output:-
        tensor([[0.0200, 0.9200, 0.0200, 0.0200, 0.0200],
                [0.0200, 0.0200, 0.9200, 0.0200, 0.0200]])
        """
        batch_size = target.shape[0]
        confidence = torch.as_tensor(batch_size * [(1.0 - smoothing)]).unsqueeze(1)
        q = torch.zeros_like(pred).fill_((self.smoothing / self.k)).scatter_(dim=1, index=target.unsqueeze(1),
                                                                             src=confidence, reduce='add')

        return q

class Loss_Inception_v3(nn.Module):
    def __init__(self, K, smoothing):
        super(Loss_Inception_v3, self).__init__()
        self.lsr = LabelSmoothing(K, smoothing)

    def forward(self, y, p):
        '''
        Params
        y: true label value --> batch_size
        p: predicted by model --> batch_size, num_classes
        Return:
        Loss values using LabelSmoothing CrossEntropy
        '''
        q_dist = self.lsr(y, p)
        p_k_x = torch.log(torch.softmax(p, dim=1))
        l = 0
        for i in range(p.shape[0]):
            l += torch.sum(p_k_x[i] * q_dist[i])

        return l

In [ ]:
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

def loss_batch(loss_func, outputs, target, opt=None):
    if np.shape(outputs)[0] == 2:
        output, aux_out = outputs

        output_loss = loss_func(output, target)
        aux_out_loss = loss_func(aux_out, target)

        loss = output_loss + 0.3*aux_out_loss

    else:
        loss = loss_func(outputs, target)

    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    return loss.item()

In [ ]:
model = GoogLeNet_v3(num_classes=num_classes).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=8, gamma=0.96)

criterion = torch.nn.CrossEntropyLoss(reduction='sum').cuda()

In [ ]:
best_acc = 0
for epoch in range(epochs):
    current_lr = get_lr(optimizer)
    print("\n----- epoch: {}/{}, lr: {} -----".format(
        epoch+1, epochs, optimizer.param_groups[0]["lr"]))

    # train for one epoch
    model.aux_logits = True
    start_time = time.time()
    train(train_loader, epoch, model, optimizer, criterion)

    # auxiliary classifier is not used when testing.
    model.aux_logits = False
    test_acc = test(test_loader,epoch,model)

    elapsed_time = time.time() - start_time
    print('==> {:.2f} seconds to train this epoch\n'.format(elapsed_time))
    # learning rate scheduling
    scheduler.step()
    
    # Save model for best accuracy
    if best_acc < test_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), path2weights)

torch.save(model.state_dict(), path2weights)
print(f"Best Top-1 Accuracy: {best_acc}")


----- epoch: 0/100, lr: 0.001 -----
Epoch: [0][  0/782]	Time  1.843 ( 1.843)	Loss 3.8375e+02 (3.8375e+02)	Acc@1   4.69 (  4.69)	Acc@5   6.25 (  6.25)
Epoch: [0][100/782]	Time  0.443 ( 0.453)	Loss 3.3549e+12 (1.6918e+13)	Acc@1   1.56 (  1.18)	Acc@5   1.56 (  5.31)
Epoch: [0][200/782]	Time  0.453 ( 0.451)	Loss 1.2315e+20 (2.0173e+20)	Acc@1   0.00 (  1.19)	Acc@5   6.25 (  5.32)
Epoch: [0][300/782]	Time  0.462 ( 0.453)	Loss 2.4936e+17 (2.0816e+20)	Acc@1   1.56 (  1.07)	Acc@5   6.25 (  5.01)
Epoch: [0][400/782]	Time  0.466 ( 0.456)	Loss 6.0850e+16 (1.5628e+20)	Acc@1   1.56 (  1.07)	Acc@5   3.12 (  4.96)
Epoch: [0][500/782]	Time  0.471 ( 0.458)	Loss 3.3259e+16 (1.2509e+20)	Acc@1   0.00 (  1.05)	Acc@5   6.25 (  5.00)
Epoch: [0][600/782]	Time  0.471 ( 0.460)	Loss 1.7347e+16 (1.0428e+20)	Acc@1   3.12 (  0.99)	Acc@5   3.12 (  4.90)
Epoch: [0][700/782]	Time  0.471 ( 0.462)	Loss 1.4277e+16 (8.9410e+19)	Acc@1   0.00 (  0.99)	Acc@5   4.69 (  4.97)
==> Train Accuracy: Acc@1 0.976 || Acc@5 5.022
==> 